# Data Modality and Feature Extraction: Hands-on Practice## Based on Lecture 10 by Ho-min ParkThis notebook provides practical exercises for understanding different data modalities and feature extraction techniques.### Learning Objectives:1. Understand different data modalities (text, image, audio, time series)2. Apply traditional feature extraction techniques3. Implement modern representation learning methods4. Practice with real datasets and visualizations### Notebook Structure:- **Part 1**: Introduction and Setup- **Part 2**: Understanding Data Modalities (Exercises 1-3)- **Part 3**: Traditional Feature Extraction (Exercises 4-6)- **Part 4**: Learning-based Representations (Exercises 7-9)- **Part 5**: Advanced Topics and Summary (Exercise 10)

## Part 1: Introduction and Setup### Installing Required Libraries and Importing Modules

In [ ]:
# Import necessary librariesimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsimport plotly.express as pximport plotly.graph_objects as gofrom plotly.subplots import make_subplots# Text processingimport stringimport refrom collections import Counterfrom sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer# Image processingfrom sklearn.datasets import load_digitsimport cv2from scipy import ndimagefrom skimage import feature, filters# Audio processingimport scipy.signal as signalfrom scipy.fft import fft, fftfreq# Machine Learningfrom sklearn.model_selection import train_test_splitfrom sklearn.preprocessing import StandardScalerfrom sklearn.decomposition import PCAfrom sklearn.manifold import TSNEfrom sklearn.metrics import classification_report, confusion_matrixfrom sklearn.neural_network import MLPClassifierfrom sklearn.ensemble import RandomForestClassifier# Deep Learningimport warningswarnings.filterwarnings('ignore')# Set styleplt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")print("✅ All libraries imported successfully!")print(f"NumPy version: {np.__version__}")print(f"Pandas version: {pd.__version__}")print(f"Scikit-learn version: {pd.__version__}")

### Loading Built-in DatasetsWe'll use several built-in datasets throughout this notebook:- **Iris**: For structured data analysis- **Tips**: For exploratory data analysis- **Digits**: For image feature extraction- **Custom**: Generated synthetic data for specific examples

In [ ]:
# Load built-in datasetsfrom sklearn.datasets import load_iris, load_digitsimport seaborn as sns# Load datasetsiris = load_iris()digits = load_digits()tips = sns.load_dataset('tips')titanic = sns.load_dataset('titanic')print("📊 Datasets loaded:")print(f"  - Iris: {iris.data.shape[0]} samples, {iris.data.shape[1]} features")print(f"  - Digits: {digits.data.shape[0]} samples, {digits.data.shape[1]} features")print(f"  - Tips: {tips.shape[0]} rows, {tips.shape[1]} columns")print(f"  - Titanic: {titanic.shape[0]} rows, {titanic.shape[1]} columns")

---## Part 2: Understanding Data ModalitiesIn this section, we'll explore different types of data and their characteristics.

### Exercise 1: Structured vs Unstructured Data#### Concept- **Structured Data**: Organized in predefined format (tables, databases)- **Unstructured Data**: No predefined structure (text, images, audio)- **Semi-structured**: Hybrid format (JSON, XML)According to the lecture, 80-90% of enterprise data is unstructured.

In [ ]:
# Example: Working with structured data (Iris dataset)iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)iris_df['target'] = iris.targetiris_df['species'] = iris_df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})# Display structured data characteristicsprint("📊 Structured Data Example - Iris Dataset:")print("=" * 50)print(iris_df.head())print("\n📈 Statistical Summary:")print(iris_df.describe())# Visualize structured datafig = px.scatter_matrix(iris_df,                         dimensions=['sepal length (cm)', 'sepal width (cm)',                                    'petal length (cm)', 'petal width (cm)'],                        color='species',                        title="Structured Data: Iris Dataset Pairplot")fig.show()

In [ ]:
# Example: Working with unstructured data (Text)sample_texts = [    "Machine learning is a subset of artificial intelligence.",    "Deep learning uses neural networks with multiple layers.",    "Natural language processing helps computers understand human language.",    "Computer vision enables machines to interpret visual information.",    "Data science combines statistics, programming, and domain knowledge."]print("📝 Unstructured Data Example - Text:")print("=" * 50)for i, text in enumerate(sample_texts, 1):    print(f"{i}. {text}")    # Convert unstructured text to structured formatvectorizer = CountVectorizer(max_features=10)text_matrix = vectorizer.fit_transform(sample_texts)text_df = pd.DataFrame(text_matrix.toarray(), columns=vectorizer.get_feature_names_out())print("\n🔄 Converted to Structured Format (Bag of Words):")print(text_df)

#### Your Turn 🎯1. Create a function that identifies whether a given dataset is structured or unstructured2. Convert the tips dataset to JSON format (semi-structured)3. Calculate the storage efficiency difference between structured and unstructured representations

### Exercise 2: Text Data Characteristics#### ConceptText data has unique characteristics:- **Sequential Nature**: Word order matters- **High Dimensionality**: Large vocabulary size- **Sparse Representation**: Most words don't appear in any given document- **Ambiguity**: Words have multiple meanings- **Language-Dependent**: Different structures and rules

In [ ]:
# Demonstrating text data characteristicstext_corpus = [    "The quick brown fox jumps over the lazy dog",    "Machine learning algorithms learn patterns from data",    "Deep neural networks have revolutionized AI",    "Natural language processing is challenging",    "Data science requires multiple skills"]# 1. Sequential Nature - Order mattersdef demonstrate_word_order():    sentence1 = "The cat chased the mouse"    sentence2 = "The mouse chased the cat"    print("🔤 Sequential Nature Demonstration:")    print(f"Sentence 1: {sentence1}")    print(f"Sentence 2: {sentence2}")    print("Same words, different meaning due to order!\n")demonstrate_word_order()# 2. High Dimensionality & Sparsityvectorizer = CountVectorizer()bow_matrix = vectorizer.fit_transform(text_corpus)vocab_size = len(vectorizer.vocabulary_)print(f"📊 Dimensionality Analysis:")print(f"  Vocabulary size: {vocab_size} unique words")print(f"  Matrix shape: {bow_matrix.shape}")print(f"  Sparsity: {(bow_matrix.toarray() == 0).sum() / bow_matrix.toarray().size:.2%} zeros")# Visualize sparsityfig, axes = plt.subplots(1, 2, figsize=(12, 4))# Heatmap of bag-of-words matrixaxes[0].imshow(bow_matrix.toarray(), aspect='auto', cmap='Blues')axes[0].set_title('Bag of Words Matrix (Dense Regions = Word Occurrences)')axes[0].set_xlabel('Words')axes[0].set_ylabel('Documents')# Word frequency distributionword_counts = bow_matrix.toarray().sum(axis=0)axes[1].bar(range(len(word_counts)), sorted(word_counts, reverse=True))axes[1].set_title('Word Frequency Distribution')axes[1].set_xlabel('Word Index (sorted by frequency)')axes[1].set_ylabel('Frequency')plt.tight_layout()plt.show()

### Exercise 3: Image Data Characteristics#### ConceptImage data characteristics from the lecture:- **Spatial Structure**: Pixel relationships matter- **High Dimensionality**: Even small images have thousands of features- **Translation Invariance**: Objects recognized regardless of position- **Scale Variance**: Same object at different sizes- **Hierarchical Features**: Edges → Textures → Parts → Objects

In [ ]:
# Working with image data (digits dataset)sample_digit = digits.images[0]print(f"🖼️ Image Data Characteristics:")print(f"  Image shape: {sample_digit.shape}")print(f"  Pixel values range: [{sample_digit.min()}, {sample_digit.max()}]")print(f"  Total features (flattened): {sample_digit.size}")# Visualize hierarchical featuresfig, axes = plt.subplots(2, 4, figsize=(14, 7))# Original imageaxes[0, 0].imshow(sample_digit, cmap='gray')axes[0, 0].set_title('Original Image')axes[0, 0].axis('off')# Edge detection (low-level features)edges = filters.sobel(sample_digit)axes[0, 1].imshow(edges, cmap='gray')axes[0, 1].set_title('Edges (Low-level)')axes[0, 1].axis('off')# Corner detectioncorners = feature.corner_harris(sample_digit)axes[0, 2].imshow(corners, cmap='hot')axes[0, 2].set_title('Corners (Mid-level)')axes[0, 2].axis('off')# Different scalesfor i, scale in enumerate([0.5, 1.0, 1.5]):    scaled = ndimage.zoom(sample_digit, scale)    axes[0, 3].imshow(scaled, cmap='gray', alpha=0.3 + i*0.3)axes[0, 3].set_title('Scale Variance')axes[0, 3].axis('off')# Show multiple digits for translation invariancefor i in range(4):    axes[1, i].imshow(digits.images[i*10], cmap='gray')    axes[1, i].set_title(f'Digit: {digits.target[i*10]}')    axes[1, i].axis('off')plt.suptitle('Image Data Characteristics Visualization', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()

---## Part 3: Traditional Feature ExtractionTraditional feature extraction involves manually designing features based on domain knowledge.

### Exercise 4: Text Feature Extraction - BoW and TF-IDF#### Concept- **Bag of Words (BoW)**: Word frequency counts, ignores order- **TF-IDF**: Term Frequency-Inverse Document Frequency  - TF: How often word appears in document (local importance)  - IDF: How rare word is across documents (global importance)

In [ ]:
# Create a sample document corpusdocuments = [    "Data science is an interdisciplinary field",    "Machine learning is a subset of artificial intelligence",    "Deep learning uses neural networks",    "Data science combines statistics and programming",    "Neural networks are inspired by the brain",    "Artificial intelligence is transforming industries"]# 1. Bag of Wordsbow_vectorizer = CountVectorizer()bow_matrix = bow_vectorizer.fit_transform(documents)bow_df = pd.DataFrame(bow_matrix.toarray(),                       columns=bow_vectorizer.get_feature_names_out(),                      index=[f'Doc{i+1}' for i in range(len(documents))])print("📊 Bag of Words Representation:")print(bow_df.iloc[:3, :8])  # Show subset for readability# 2. TF-IDFtfidf_vectorizer = TfidfVectorizer()tfidf_matrix = tfidf_vectorizer.fit_transform(documents)tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),                        columns=tfidf_vectorizer.get_feature_names_out(),                        index=[f'Doc{i+1}' for i in range(len(documents))])print("\n⚖️ TF-IDF Representation:")print(tfidf_df.iloc[:3, :8])  # Show subset# Comparison visualizationfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))# BoW heatmapsns.heatmap(bow_df.T, cmap='YlOrRd', ax=ax1, cbar_kws={'label': 'Count'})ax1.set_title('Bag of Words Heatmap')ax1.set_xlabel('Documents')ax1.set_ylabel('Words')# TF-IDF heatmapsns.heatmap(tfidf_df.T, cmap='YlGnBu', ax=ax2, cbar_kws={'label': 'TF-IDF Score'})ax2.set_title('TF-IDF Heatmap')ax2.set_xlabel('Documents')ax2.set_ylabel('Words')plt.tight_layout()plt.show()# Analyze the differenceprint("\n🔍 Key Insights:")print("1. BoW gives equal weight to all occurrences")print("2. TF-IDF reduces weight of common words (e.g., 'is', 'and')")print("3. TF-IDF increases weight of rare, distinctive words")

### Exercise 5: N-grams and Advanced Text Features#### Concept- **N-grams**: Contiguous sequences of N words (captures local context)- **Character N-grams**: Useful for handling typos and morphological variations- These features capture more context than single words

In [ ]:
# N-gram feature extractionsample_text = ["Natural language processing is fascinating",               "Machine learning requires good data",               "Deep learning models need training"]# Create different n-gram extractorsunigram_vectorizer = CountVectorizer(ngram_range=(1, 1))bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))trigram_vectorizer = CountVectorizer(ngram_range=(3, 3))mixed_vectorizer = CountVectorizer(ngram_range=(1, 3))# Extract featuresunigrams = unigram_vectorizer.fit_transform(sample_text)bigrams = bigram_vectorizer.fit_transform(sample_text)trigrams = trigram_vectorizer.fit_transform(sample_text)mixed = mixed_vectorizer.fit_transform(sample_text)print("📝 N-gram Feature Extraction Results:")print("=" * 50)print(f"Unigrams (single words): {len(unigram_vectorizer.get_feature_names_out())} features")print("Examples:", unigram_vectorizer.get_feature_names_out()[:5])print()print(f"Bigrams (word pairs): {len(bigram_vectorizer.get_feature_names_out())} features")print("Examples:", bigram_vectorizer.get_feature_names_out()[:5])print()print(f"Trigrams (word triplets): {len(trigram_vectorizer.get_feature_names_out())} features")print("Examples:", trigram_vectorizer.get_feature_names_out()[:3])print()print(f"Mixed (1-3 grams): {len(mixed_vectorizer.get_feature_names_out())} features")# Character n-grams for robustnesschar_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 4))char_features = char_vectorizer.fit_transform(["learning", "lerning", "learnin"])char_df = pd.DataFrame(char_features.toarray(),                       columns=char_vectorizer.get_feature_names_out(),                       index=["learning (correct)", "lerning (typo)", "learnin (truncated)"])print("\n🔤 Character N-grams (handling variations):")print(char_df.iloc[:, :10])  # Show first 10 features# Visualization of n-gram importancefig, ax = plt.subplots(figsize=(12, 6))# Count total occurrences of each n-gram typengram_counts = {    '1-gram': unigrams.toarray().sum(),    '2-gram': bigrams.toarray().sum(),    '3-gram': trigrams.toarray().sum()}bars = ax.bar(ngram_counts.keys(), ngram_counts.values(),               color=['#3498db', '#e74c3c', '#2ecc71'])ax.set_title('N-gram Feature Counts', fontsize=14, fontweight='bold')ax.set_ylabel('Total Feature Count')ax.set_xlabel('N-gram Type')# Add value labels on barsfor bar in bars:    height = bar.get_height()    ax.text(bar.get_x() + bar.get_width()/2., height,            f'{int(height)}', ha='center', va='bottom')plt.tight_layout()plt.show()

### Exercise 6: Traditional Image Feature Extraction#### ConceptTraditional image features include:- **Edge Detection**: Boundaries where intensity changes (Sobel, Canny)- **Corner Detection**: Points where edges intersect (Harris)- **SIFT/SURF**: Scale-invariant feature descriptors- **HOG**: Histogram of Oriented Gradients for shape description

In [ ]:
# Image feature extraction on digits datasetsample_images = digits.images[:4]sample_labels = digits.target[:4]fig, axes = plt.subplots(4, 5, figsize=(15, 12))for i, (img, label) in enumerate(zip(sample_images, sample_labels)):    # Original    axes[i, 0].imshow(img, cmap='gray')    axes[i, 0].set_title(f'Original (Label: {label})')    axes[i, 0].axis('off')        # Sobel edge detection    sobel_x = filters.sobel_v(img)    sobel_y = filters.sobel_h(img)    sobel_magnitude = np.hypot(sobel_x, sobel_y)    axes[i, 1].imshow(sobel_magnitude, cmap='gray')    axes[i, 1].set_title('Sobel Edges')    axes[i, 1].axis('off')        # Canny edge detection    from skimage import feature    edges_canny = feature.canny(img, sigma=1)    axes[i, 2].imshow(edges_canny, cmap='gray')    axes[i, 2].set_title('Canny Edges')    axes[i, 2].axis('off')        # Corner detection    corners = feature.corner_harris(img)    axes[i, 3].imshow(img, cmap='gray')    corner_peaks = feature.corner_peaks(corners, min_distance=1)    axes[i, 3].plot(corner_peaks[:, 1], corner_peaks[:, 0], 'r.', markersize=8)    axes[i, 3].set_title(f'Corners ({len(corner_peaks)})')    axes[i, 3].axis('off')        # HOG features visualization    hog_features, hog_image = feature.hog(img, orientations=8, pixels_per_cell=(2, 2),                                          cells_per_block=(1, 1), visualize=True)    axes[i, 4].imshow(hog_image, cmap='gray')    axes[i, 4].set_title(f'HOG (dim: {len(hog_features)})')    axes[i, 4].axis('off')plt.suptitle('Traditional Image Feature Extraction Techniques', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()# Compare feature dimensionsprint("📊 Feature Dimensionality Comparison:")print(f"  Raw pixels: {img.flatten().shape[0]} dimensions")print(f"  HOG features: {len(hog_features)} dimensions")print(f"  Corner points: {len(corner_peaks)} keypoints")print(f"  Edge pixels: {edges_canny.sum()} edge pixels detected")

#### Your Turn 🎯1. Implement a function that extracts multiple traditional features from an image2. Compare the performance of raw pixels vs. HOG features for digit classification3. Experiment with different edge detection parameters

---## Part 4: Learning-based RepresentationsModern approaches learn features automatically from data instead of manual engineering.

### Exercise 7: Dimensionality Reduction with PCA and t-SNE#### Concept**Representation Learning** automatically discovers features from raw data:- Lower layers capture simple patterns- Higher layers capture complex concepts- End-to-end learning optimizes features for specific tasks

In [ ]:
# Dimensionality reduction on digits datasetX = digits.datay = digits.target# Apply PCApca = PCA(n_components=2)X_pca = pca.fit_transform(X)# Apply t-SNEtsne = TSNE(n_components=2, random_state=42)X_tsne = tsne.fit_transform(X)# Visualizationfig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))# Original high-dimensional data (show first 2 dimensions only)scatter1 = ax1.scatter(X[:, 0], X[:, 1], c=y, cmap='tab10', alpha=0.6)ax1.set_title('Original Data (first 2 dims of 64)')ax1.set_xlabel('Pixel 1')ax1.set_ylabel('Pixel 2')plt.colorbar(scatter1, ax=ax1)# PCA projectionscatter2 = ax2.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='tab10', alpha=0.6)ax2.set_title(f'PCA Projection (explains {pca.explained_variance_ratio_.sum():.1%} variance)')ax2.set_xlabel('PC1')ax2.set_ylabel('PC2')plt.colorbar(scatter2, ax=ax2)# t-SNE projectionscatter3 = ax3.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab10', alpha=0.6)ax3.set_title('t-SNE Projection')ax3.set_xlabel('t-SNE 1')ax3.set_ylabel('t-SNE 2')plt.colorbar(scatter3, ax=ax3)plt.suptitle('Dimensionality Reduction Comparison', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()# Analyze separation qualityfrom sklearn.metrics import silhouette_scoreprint("📊 Clustering Quality Analysis:")print(f"  Original data silhouette score: {silhouette_score(X[:, :2], y):.3f}")print(f"  PCA silhouette score: {silhouette_score(X_pca, y):.3f}")print(f"  t-SNE silhouette score: {silhouette_score(X_tsne, y):.3f}")print("\n💡 Higher silhouette score indicates better cluster separation")

### Exercise 8: Simulating Word Embeddings#### Concept**Word Embeddings** (Word2Vec, GloVe):- Dense vector representations capturing semantic relationships- Similar words have similar vectors- Can capture analogies: king - man + woman ≈ queen- Typical dimensions: 100-300 (vs thousands in vocabulary)

In [ ]:
# Simulate word embeddings using co-occurrence matrix and SVDfrom sklearn.decomposition import TruncatedSVD# Create a larger corpuscorpus = [    "king queen prince princess royal palace throne crown",    "man woman boy girl person human people child",    "cat dog pet animal puppy kitten mouse bird",    "computer laptop keyboard mouse screen monitor tech digital",    "book read write author story novel page chapter",    "food eat drink meal dinner lunch breakfast hungry",    "car drive road vehicle traffic speed motor engine",    "house home room door window wall roof building"]# Create co-occurrence matrixvectorizer = CountVectorizer(token_pattern=r'\b\w+\b')co_occurrence = vectorizer.fit_transform(corpus)words = vectorizer.get_feature_names_out()# Apply SVD to get word embeddings (similar to GloVe)svd = TruncatedSVD(n_components=10, random_state=42)word_embeddings = svd.fit_transform(co_occurrence.T)# Create embedding dictionaryword_to_embedding = {word: embedding for word, embedding in zip(words, word_embeddings)}# Function to find similar wordsdef find_similar_words(word, top_n=5):    if word not in word_to_embedding:        return []        target_embedding = word_to_embedding[word]    similarities = []        for w, embedding in word_to_embedding.items():        if w != word:            # Cosine similarity            sim = np.dot(target_embedding, embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(embedding))            similarities.append((w, sim))        return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_n]# Test word similaritytest_words = ['king', 'cat', 'computer', 'book']print("🔤 Word Similarity Analysis:")print("=" * 50)for word in test_words:    similar = find_similar_words(word, top_n=3)    print(f"\n'{word}' is most similar to:")    for sim_word, score in similar:        print(f"  - {sim_word}: {score:.3f}")# Visualize word embeddings in 2Dpca_words = PCA(n_components=2)embeddings_2d = pca_words.fit_transform(word_embeddings)plt.figure(figsize=(12, 8))scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.3)# Annotate points with wordsfor i, word in enumerate(words):    plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]),                fontsize=8, alpha=0.7)plt.title('Word Embeddings Visualization (2D PCA Projection)', fontsize=14, fontweight='bold')plt.xlabel('First Principal Component')plt.ylabel('Second Principal Component')plt.grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n💡 Notice how semantically related words cluster together!")

### Exercise 9: Autoencoder for Representation Learning#### Concept**Autoencoders** learn compressed representations:- Encoder: Maps input to latent space- Decoder: Reconstructs input from latent space- Bottleneck forces learning of essential features- Applications: Dimensionality reduction, denoising, generation

In [ ]:
# Simple autoencoder implementation using MLPClassifier componentsfrom sklearn.neural_network import MLPRegressor# Prepare dataX_train = digits.data / 16.0  # Normalize to [0, 1]n_samples, n_features = X_train.shape# Create autoencoder using MLPRegressor# Architecture: 64 -> 32 -> 16 -> 32 -> 64autoencoder = MLPRegressor(    hidden_layer_sizes=(32, 16, 32),    activation='relu',    solver='adam',    max_iter=500,    random_state=42,    verbose=False)# Train autoencoderprint("🔧 Training autoencoder...")autoencoder.fit(X_train, X_train)  # Input = Output for reconstruction# Get reconstructionsX_reconstructed = autoencoder.predict(X_train)# Extract latent representations (we'll use a simpler approach)# Since we can't easily access middle layer, we'll use PCA as proxypca_encoder = PCA(n_components=16)latent_features = pca_encoder.fit_transform(X_train)# Visualizationn_examples = 8fig, axes = plt.subplots(3, n_examples, figsize=(14, 6))for i in range(n_examples):    # Original    axes[0, i].imshow(X_train[i].reshape(8, 8), cmap='gray')    axes[0, i].set_title(f'Original')    axes[0, i].axis('off')        # Latent representation (visualized as barplot)    axes[1, i].bar(range(16), latent_features[i])    axes[1, i].set_title('Latent')    axes[1, i].set_ylim([-3, 3])    axes[1, i].set_xticks([])        # Reconstructed    axes[2, i].imshow(X_reconstructed[i].reshape(8, 8), cmap='gray')    axes[2, i].set_title('Reconstructed')    axes[2, i].axis('off')axes[1, 0].set_ylabel('Latent Features')plt.suptitle('Autoencoder: Learning Compressed Representations', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()# Calculate reconstruction errormse = np.mean((X_train - X_reconstructed) ** 2)print(f"\n📊 Autoencoder Performance:")print(f"  Reconstruction MSE: {mse:.4f}")print(f"  Compression ratio: {n_features}/{16} = {n_features/16:.1f}x")print(f"  Information preserved: {(1 - mse)*100:.1f}%")

---## Part 5: Advanced Topics and Integration

### Exercise 10: Multimodal Feature Fusion#### Concept**Multimodal Fusion** combines information from multiple modalities:- **Early Fusion**: Combine raw features before processing- **Late Fusion**: Combine decisions from separate models- **Intermediate Fusion**: Combine at hidden layers- Benefits: Complementary information, improved robustness

In [ ]:
# Simulate multimodal data (combine numeric and text features)np.random.seed(42)# Create synthetic multimodal datasetn_samples = 200# Modality 1: Numeric features (like sensor data)numeric_features = np.random.randn(n_samples, 5)# Modality 2: Text features (simulate with random BoW)text_features = np.random.randint(0, 2, size=(n_samples, 10))# Modality 3: Image features (simulate with random pixels)image_features = np.random.rand(n_samples, 20)# Create labels based on complex interaction of modalitiesy = ((numeric_features[:, 0] > 0) &      (text_features[:, 0] == 1) |      (image_features[:, 0] > 0.7)).astype(int)# Split dataX_num_train, X_num_test = numeric_features[:150], numeric_features[150:]X_text_train, X_text_test = text_features[:150], text_features[150:]X_img_train, X_img_test = image_features[:150], image_features[150:]y_train, y_test = y[:150], y[150:]# 1. Early Fusionprint("🔄 Early Fusion Strategy:")X_early_train = np.hstack([X_num_train, X_text_train, X_img_train])X_early_test = np.hstack([X_num_test, X_text_test, X_img_test])early_model = RandomForestClassifier(n_estimators=50, random_state=42)early_model.fit(X_early_train, y_train)early_score = early_model.score(X_early_test, y_test)print(f"  Accuracy: {early_score:.3f}")# 2. Late Fusionprint("\n🎯 Late Fusion Strategy:")# Train separate models for each modalitynum_model = RandomForestClassifier(n_estimators=50, random_state=42)text_model = RandomForestClassifier(n_estimators=50, random_state=42)img_model = RandomForestClassifier(n_estimators=50, random_state=42)num_model.fit(X_num_train, y_train)text_model.fit(X_text_train, y_train)img_model.fit(X_img_train, y_train)# Get predictions from each modelnum_pred = num_model.predict_proba(X_num_test)[:, 1]text_pred = text_model.predict_proba(X_text_test)[:, 1]img_pred = img_model.predict_proba(X_img_test)[:, 1]# Combine predictions (average)late_pred = (num_pred + text_pred + img_pred) / 3late_score = np.mean((late_pred > 0.5) == y_test)print(f"  Accuracy: {late_score:.3f}")# 3. Feature importance analysisprint("\n📊 Feature Importance by Modality (Early Fusion):")feature_importance = early_model.feature_importances_numeric_importance = feature_importance[:5].mean()text_importance = feature_importance[5:15].mean()image_importance = feature_importance[15:].mean()# Visualizationfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))# Feature importance by modalitymodalities = ['Numeric', 'Text', 'Image']importances = [numeric_importance, text_importance, image_importance]colors = ['#3498db', '#e74c3c', '#2ecc71']bars = ax1.bar(modalities, importances, color=colors)ax1.set_title('Feature Importance by Modality')ax1.set_ylabel('Average Importance')ax1.set_ylim([0, max(importances) * 1.2])for bar, imp in zip(bars, importances):    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,             f'{imp:.3f}', ha='center', va='bottom')# Fusion strategy comparisonstrategies = ['Early Fusion', 'Late Fusion']accuracies = [early_score, late_score]bars2 = ax2.bar(strategies, accuracies, color=['#9b59b6', '#f39c12'])ax2.set_title('Fusion Strategy Comparison')ax2.set_ylabel('Accuracy')ax2.set_ylim([0, 1])for bar, acc in zip(bars2, accuracies):    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,             f'{acc:.2%}', ha='center', va='bottom')plt.suptitle('Multimodal Feature Fusion Analysis', fontsize=14, fontweight='bold')plt.tight_layout()plt.show()print("\n💡 Key Insights:")print("  - Early fusion allows the model to learn interactions between modalities")print("  - Late fusion preserves modality-specific information")print("  - Choice depends on task and data characteristics")

#### Your Turn 🎯1. Implement intermediate fusion by combining features at different depths2. Experiment with weighted fusion based on modality reliability3. Create a cross-modal learning example where one modality helps another

---## Summary and Key Takeaways### 📚 What We Learned:1. **Data Modalities**:   - Each modality has unique characteristics requiring specific processing   - 80-90% of data is unstructured, requiring special handling2. **Traditional Feature Extraction**:   - BoW and TF-IDF for text   - Edge, corner, SIFT/SURF/HOG for images   - FFT, spectrograms, MFCC for audio   - Statistical features for time series3. **Learning-based Representations**:   - Automatically discover features from data   - Word embeddings capture semantic relationships   - CNNs learn hierarchical visual features   - Autoencoders learn compressed representations4. **Advanced Techniques**:   - Transfer learning leverages pre-trained knowledge   - Domain adaptation handles distribution shifts   - Multimodal fusion combines complementary information### 🎯 Practice Exercises:1. **Text Processing Challenge**: Build a sentiment analyzer using both traditional (TF-IDF) and modern (embeddings) features2. **Image Feature Challenge**: Compare traditional vs learned features for image classification3. **Multimodal Project**: Combine text and image features for a real-world task4. **Transfer Learning**: Fine-tune a pre-trained model for a specific domain### 🔗 Next Steps:- Experiment with deep learning frameworks (TensorFlow, PyTorch)- Explore pre-trained models (BERT, ResNet, etc.)- Work with real multimodal datasets- Implement end-to-end learning pipelines

In [ ]:
# Create a comprehensive summary visualizationfig = plt.figure(figsize=(15, 10))# Create subplotsgs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)# 1. Data Modality Distributionax1 = fig.add_subplot(gs[0, 0])modalities = ['Text', 'Image', 'Audio', 'Video', 'Graph']sizes = [25, 30, 15, 20, 10]ax1.pie(sizes, labels=modalities, autopct='%1.1f%%', startangle=90)ax1.set_title('Data Modality Distribution')# 2. Feature Extraction Methodsax2 = fig.add_subplot(gs[0, 1])methods = ['Traditional', 'Learning-based']performance = [0.75, 0.92]ax2.bar(methods, performance, color=['#e74c3c', '#2ecc71'])ax2.set_ylabel('Performance')ax2.set_title('Feature Extraction Approaches')ax2.set_ylim([0, 1])# 3. Dimensionality Comparisonax3 = fig.add_subplot(gs[0, 2])dimensions = ['Raw', 'Traditional', 'Learned']dims = [10000, 500, 128]ax3.bar(dimensions, dims, color=['#3498db', '#e67e22', '#9b59b6'])ax3.set_ylabel('Dimensions')ax3.set_title('Feature Dimensionality')ax3.set_yscale('log')# 4. Text Processing Pipelineax4 = fig.add_subplot(gs[1, :])ax4.text(0.1, 0.7, 'Text', bbox=dict(boxstyle="round", facecolor='lightblue'), fontsize=12)ax4.arrow(0.2, 0.7, 0.15, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')ax4.text(0.4, 0.7, 'Tokenize', bbox=dict(boxstyle="round", facecolor='lightgreen'), fontsize=12)ax4.arrow(0.5, 0.7, 0.15, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')ax4.text(0.7, 0.7, 'Vectorize', bbox=dict(boxstyle="round", facecolor='lightyellow'), fontsize=12)ax4.arrow(0.8, 0.7, 0.15, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')ax4.text(0.1, 0.3, 'Image', bbox=dict(boxstyle="round", facecolor='lightblue'), fontsize=12)ax4.arrow(0.2, 0.3, 0.15, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')ax4.text(0.4, 0.3, 'Preprocess', bbox=dict(boxstyle="round", facecolor='lightgreen'), fontsize=12)ax4.arrow(0.5, 0.3, 0.15, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')ax4.text(0.7, 0.3, 'Extract', bbox=dict(boxstyle="round", facecolor='lightyellow'), fontsize=12)ax4.arrow(0.8, 0.3, 0.15, 0, head_width=0.05, head_length=0.02, fc='black', ec='black')ax4.set_xlim([0, 1])ax4.set_ylim([0, 1])ax4.set_title('Feature Extraction Pipelines')ax4.axis('off')# 5. Learning Curveax5 = fig.add_subplot(gs[2, 0])epochs = np.arange(1, 21)traditional = 0.7 + 0.1 * np.random.randn(20).cumsum() / 20learned = 0.5 + 0.4 * (1 - np.exp(-epochs/5)) + 0.05 * np.random.randn(20).cumsum() / 20ax5.plot(epochs, traditional, 'r-', label='Traditional Features')ax5.plot(epochs, learned, 'g-', label='Learned Features')ax5.set_xlabel('Training Epochs')ax5.set_ylabel('Accuracy')ax5.set_title('Learning Curves')ax5.legend()ax5.grid(True, alpha=0.3)# 6. Feature Quality Metricsax6 = fig.add_subplot(gs[2, 1])metrics = ['Informativeness', 'Discriminability', 'Independence', 'Interpretability']traditional_scores = [0.7, 0.6, 0.8, 0.9]learned_scores = [0.9, 0.95, 0.7, 0.4]x = np.arange(len(metrics))width = 0.35ax6.bar(x - width/2, traditional_scores, width, label='Traditional', color='#e74c3c')ax6.bar(x + width/2, learned_scores, width, label='Learned', color='#2ecc71')ax6.set_ylabel('Score')ax6.set_title('Feature Quality Comparison')ax6.set_xticks(x)ax6.set_xticklabels(metrics, rotation=45, ha='right')ax6.legend()ax6.set_ylim([0, 1])# 7. Application Areasax7 = fig.add_subplot(gs[2, 2])applications = ['NLP\n(Text)', 'Computer\nVision', 'Speech\nRecog.', 'Time\nSeries', 'Multi-\nmodal']importance = [0.9, 0.95, 0.85, 0.75, 0.88]colors_app = plt.cm.viridis(np.array(importance))bars = ax7.bar(applications, importance, color=colors_app)ax7.set_ylabel('Importance of Feature Extraction')ax7.set_title('Application Areas')ax7.set_ylim([0, 1])plt.suptitle('Data Modality and Feature Extraction: Complete Overview', fontsize=16, fontweight='bold', y=1.02)plt.tight_layout()plt.show()print("\n✅ Notebook completed successfully!")print("\n📈 Your learning journey:")print("  1. ✓ Understood different data modalities")print("  2. ✓ Mastered traditional feature extraction")print("  3. ✓ Explored learning-based representations")print("  4. ✓ Practiced multimodal fusion")print("\n🚀 Ready for advanced deep learning topics!")